In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
# remove columns with all identical value

def identify_constant_features(dataframe):
    count_uniques = dataframe.apply(lambda x: len(x.unique()))
    constants = count_uniques[count_uniques == 1].index.tolist()
    return constants

nonUnique_feature_train = set(identify_constant_features(train))
train.drop(nonUnique_feature_train, inplace=True, axis=1)

from itertools import combinations
from numpy import array,array_equal

def identify_equal_features(dataframe):
    features_to_compare = list(combinations(dataframe.columns.tolist(),2))
    equal_features = []
    for compare in features_to_compare:
        is_equal = array_equal(dataframe[compare[0]],dataframe[compare[1]])
        if is_equal:
            equal_features.append(list(compare))
    return equal_features

equal_features_train = identify_equal_features(train)

# Remove the second feature of each pair.

features_to_drop = array(equal_features_train)[:,1] 
train.drop(features_to_drop, axis=1, inplace=True)

test.drop(nonUnique_feature_train, inplace=True, axis=1)
test.drop(features_to_drop, axis=1, inplace=True)

In [ ]:
train.loc[train["var3"]==-999999,"var3"] = 2
all_clomuns = list(train.columns.values)
all_clomuns.remove("ID")
all_clomuns.remove("TARGET")
from sklearn.feature_selection import f_classif
import numpy as np
f_values, p_values = f_classif(train[all_clomuns], train["TARGET"])
p_values = -np.log10(p_values)
p_values_dict = {}
for idx,value in enumerate(p_values):
    if value not in p_values_dict:
        p_values_dict[value] = [all_clomuns[idx]]
    else:
        p_values_dict[value].append(all_clomuns[idx])
identical_pvalues = []
for value in p_values_dict.keys():
    if (len(p_values_dict[value])>1):
        for name in p_values_dict[value][1:]:
            all_clomuns.remove(name)

In [ ]:
train2 = train[all_clomuns]
corrMatrix=train2.corr()

corrMatrix.loc[:,:] =  np.tril(corrMatrix, k=-1) # borrowed from Karl D's answer
already_in = set()
result = []
for col in corrMatrix:
    perfect_corr = corrMatrix[col][corrMatrix[col] > 0.9].index.tolist()
    if perfect_corr and col not in already_in:
        already_in.update(set(perfect_corr))
        perfect_corr.append(col)
        result.append(perfect_corr)
for pair in result:
    for name in pair[1:]:
        if name in all_clomuns:
            all_clomuns.remove(name)

In [ ]:
import xgboost as xgb
from sklearn.cross_validation import train_test_split

y_train = train['TARGET'].values
X_train = train[all_clomuns].values

id_test = test['ID']
X_test = test[all_clomuns].values

# length of dataset
len_train = len(X_train)
len_test  = len(X_test)

# classifier
clf = xgb.XGBClassifier(missing=np.nan, max_depth=6, n_estimators=500, learning_rate=0.02, nthread=4, subsample=0.75, colsample_bytree=0.75, seed=1024)

X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.3)

# fitting
clf.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc", eval_set=[(X_eval, y_eval)],verbose=1)

# predicting
y_pred= clf.predict_proba(X_test)[:,1]

submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("submission6.csv", index=False)